### portfolio statistics detail

# 초기 조건 / 데이터 임포트 / 벤치마크 계산

## 라이브러리 임포트 / 초기 조건 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [ ]:
transaction_fee = 0
etf_expenses = 0
tax_rate = 0
initial_cash = 10000

In [ ]:
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

In [ ]:
strategies = []

## 데이터 임포트

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/quant_projects/momentum_strategies/SPY.csv')[:-2] # 2021-09까지 결과 출력
closes = df.Close.to_numpy()
opens = df.Open.to_numpy()

In [ ]:
tb3_df = pd.read_csv('/content/drive/MyDrive/quant_projects/momentum_strategies/TB3MS.csv')
risk_free_rates = tb3_df.TB3MS.to_numpy() / 100

## 벤치마크 계산

In [ ]:
benchmark_shares = initial_cash // closes[0]
benchmark = closes * benchmark_shares

In [ ]:
def annualize(closes, opens, month_lag):
  annual_returns = []
  for i in range(len(closes) // 12 +1):
    if (i+1)*12 > len(closes):
      annual_returns.append(round((closes[-1] - opens[i*12-month_lag]) / opens[i*12-month_lag],4))
    elif i*12-month_lag < 0:
      annual_returns.append(round((closes[i*12+11-month_lag] - opens[0]) / opens[0], 4))
    else:
      annual_returns.append(round((closes[i*12+11-month_lag] - opens[i*12-month_lag]) / opens[i*12-month_lag],4))
  return annual_returns

In [ ]:
def statistics(name, closes, annual_returns):
  cagr = round(pow(closes[-1] / closes[0], 1/ (round(len(closes) / 12))) - 1, 2)
  mean_annual = round(np.mean(annual_returns),2)
  stddev = round(np.std(annual_returns),2)
  sharpe = round((np.mean(annual_returns)-0.005) / np.std(annual_returns),2)
  return {'name':name, 'final_balance':round(closes[-1],2), 'cagr':cagr, 'mean_annual':mean_annual, 'stddev':stddev, 'sharpe':sharpe}

## cagr

In [ ]:
closes[-1] / closes[0]

9.663955299085153

In [ ]:
round(len(closes) / 12)

29

In [ ]:
benchmark_cagr = round((pow(closes[-1] / closes[0], 1/ (round(len(closes) / 12))) - 1) * 100, 2)

## mean, stddev, sharpe

### 2011-2020 10yr. 연간수익률

In [ ]:
#2011-01부터 10년간 데이터
# closes = closes[215:] 
# opens = opens[215:]
# month_lag = 0
month_lag = 1 #1월이 아닌 2월부터 시작하므로
yearly_returns = []
for i in range(len(closes) // 12 +1):
  if (i+1)*12 > len(closes):
    yearly_returns.append(round((closes[-1] - opens[i*12-month_lag]) / opens[i*12-month_lag],4))
  elif i*12-month_lag < 0:
    yearly_returns.append(round((closes[i*12+11-month_lag] - opens[0]) / opens[0], 4))
  else:
    yearly_returns.append(round((closes[i*12+11-month_lag] - opens[i*12-month_lag]) / opens[i*12-month_lag],4))

In [ ]:
yearly_returns

[0.0597,
 -0.0221,
 0.3453,
 0.2025,
 0.305,
 0.2672,
 0.1905,
 -0.1151,
 -0.1341,
 -0.2335,
 0.2524,
 0.0817,
 0.0243,
 0.1312,
 0.0278,
 -0.3842,
 0.2322,
 0.1191,
 -0.0095,
 0.1147,
 0.2728,
 0.1172,
 -0.0122,
 0.1149,
 0.1858,
 -0.0669,
 0.3085,
 0.1556,
 0.1434]

### 1993-02부터 2021-09까지 연간수익률

In [ ]:
month_lag = 2 # 11,23,34번째가 12월이 아니라 10,22,34번째가 12월이므로
yearly_returns = []
decembers = [i for i in range(len(closes)) if i % 12 == (12-month_lag)]
for i in decembers:
  if i % 12 == (12-month_lag):
    if i-11 < 0:
      yearly_returns.append(round((closes[i]-opens[0]) / opens[0], 4))
    else:
      yearly_returns.append(round((closes[i]-opens[i-11]) / opens[i-11], 4))
if len(closes) > decembers[-1]:
  yearly_returns.append(round((closes[-1]-opens[decembers[-1]+1]) / opens[decembers[-1]+1], 4))

In [ ]:
yearly_returns

[0.0597,
 -0.0221,
 0.3453,
 0.2025,
 0.305,
 0.2672,
 0.1905,
 -0.1151,
 -0.1341,
 -0.2335,
 0.2524,
 0.0817,
 0.0243,
 0.1312,
 0.0278,
 -0.3842,
 0.2322,
 0.1191,
 -0.0095,
 0.1147,
 0.2728,
 0.1172,
 -0.0122,
 0.1149,
 0.1858,
 -0.0669,
 0.3085,
 0.1556,
 0.1434]

### 산술평균, 표준편차, sharpe ratio

In [ ]:
benchmark_mean_annual = round(np.mean(yearly_returns),2)

In [ ]:
benchmark_stddev = round(np.std(yearly_returns),2)

In [ ]:
np.mean(yearly_returns) / np.std(yearly_returns)

1.0897098534269936

In [ ]:
benchmark_sharpe = round((np.mean(yearly_returns)-0.005) / np.std(yearly_returns),2) #yahoo finance 10yr SPY sharpe ratio는 0.005를 사용한다.

In [ ]:
monthly_returns = [round((closes[i]-opens[i]) / opens[i], 4) for i in range(len(closes))]

In [ ]:
np.mean(monthly_returns)

0.009013953488372091

In [ ]:
np.std(monthly_returns) * math.sqrt(12)

0.13932446149394834

In [ ]:
(np.mean(monthly_returns)) / np.std(monthly_returns) * math.sqrt(12)

0.7763707873018653

In [ ]:
(np.mean(yearly_returns) - 0.005) / np.std(monthly_returns) / math.sqrt(12) #이렇게해야 portfolio visualizer와 비슷한 값이 나온다

0.8282171547881503

In [ ]:
np.mean(risk_free_rates)

0.022660465116279067

In [ ]:
(np.mean(yearly_returns) - np.mean(risk_free_rates)) / np.std(yearly_returns)

0.8846002458419089

In [ ]:
yf_spy = [18.25, 31.29, -4.45, 21.69,11.8, 1.34,13.53,32.21,15.84,2.06]
print((np.mean(yf_spy) - np.mean(risk_free_rates)) / np.std(yf_spy))
print((np.mean(yf_spy) - 0.005) / np.std(yf_spy))

1.2299025428827812
1.2314179364797961


In [ ]:
np.array(yf_spy[::-1]) - np.array(yearly_returns) * 100

array([3.01, 4.37, 4.93, 1.81, 2.56, 0.31, 3.11, 2.24, 0.44, 2.69])

### dictionary에 기록

In [ ]:
strategies.append({'name':'Benchmark', 'final_balance':round(benchmark[-1],2), 'cagr':benchmark_cagr, 'mean_annual':benchmark_mean_annual, 'stddev':benchmark_stddev, 'sharpe':benchmark_sharpe})

In [ ]:
pd.DataFrame(strategies)

,name,final_balance,cagr,mean_annual,stddev,sharpe
0,Benchmark,96556.503375,8.14,0.120391,0.11048,1.044453
